In [1]:
%matplotlib inline
%load_ext tensorboard
#!pip install keras-tuner
import pandas as pd
import numpy as np
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.layers import Layer
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)
from matplotlib import pyplot as plt
import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.python.ops.numpy_ops import np_config

2024-07-02 09:59:41.052704: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-02 09:59:41.089179: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 09:59:41.673924: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
genotypeFile = 'genotype.csv'
genotype = pd.read_csv(genotypeFile, sep = '\t', index_col = 0)
print('genotypeFile shape:', genotype.shape )

phenotypeFile = 'phenotype.csv'
multi_pheno = pd.read_csv(phenotypeFile, sep = ',', index_col = 0)
print('Phenotype_Multi shape:', multi_pheno.shape )


# take a small part to test code
# genotype
X = genotype
#display(X.head())
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno

#Chage the index for each model here accoding to the correspinding model
Y = multi_pheno.iloc[:, 2]#index=1 --> 1_Diamide_1
# Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones
# corrupted_X.head()

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
# corrupted_X_onehot.shape

# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

genotypeFile shape: (4390, 28220)
Phenotype_Multi shape: (4390, 20)


In [3]:
x_test.shape

(424, 28220, 3)

In [4]:
x_train.shape

(3452, 28220, 3)

In [5]:
len(x_train[:100])

100

In [6]:
background_list = list([i for i in range(404)])[0:100]
if len(background_list)==0:
    background_list = x_train[np.random.choice(x_train.shape[0],100, replace=False)]
background = x_train[background_list]
background.shape

(100, 28220, 3)

In [7]:
import shap
from shap import initjs
initjs()
custom_objects = None
# Define custom objects if necessary
custom_objects = None

# Load the model with specified reduction strategy
model_mse = tf.keras.models.load_model('1_Diamide_1.h5', custom_objects=custom_objects, compile=False)



The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
2024-07-02 10:00:52.935451: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10941 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:19:00.0, compute capability: 7.5
2024-07-02 10:00:52.939903: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13521 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:1a:00.0, compute capability: 7.5
2024-07-02 10:00:52.940470: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 13521 MB memory:  -> device: 2, name: Quadro RTX 5000, pci bus i

In [8]:
model_mse.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28220, 3)]        0         
                                                                 
 conv1d_4 (Conv1D)           (None, 9405, 8)           176       
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 4702, 8)          0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 1566, 8)           456       
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 783, 8)           0         
 1D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 6264)              0   

In [9]:
explainer = shap.DeepExplainer(model_mse,x_test)
base_value = explainer.expected_value

keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode. See PR #1483 for discussion.
2024-07-02 10:01:05.530355: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2024-07-02 10:01:07.229891: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [10]:
base_values = base_value[0]

In [11]:
shap_values = explainer.shap_values(x_test,check_additivity=False)

In [12]:
mean_shap_values = np.mean(shap_values, axis=0)

In [13]:
abs_shap_values = np.abs(shap_values[0])
feature_importance = np.mean(abs_shap_values, axis=0)
len(feature_importance)

28220

In [14]:
feature_importance.shape

(28220, 3)

In [15]:
feature_impportance_sum = feature_importance.sum(axis=1)
feature_impportance_sum

array([2.68212058e-06, 1.29353757e-06, 1.57545554e-06, ...,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [22]:
feature_importance_df = pd.DataFrame({
    'Feature': X.columns.tolist(),  # Convert X.columns to a list
    'Importance': feature_impportance_sum.tolist()
})

#feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
type(feature_importance_df)

pandas.core.frame.DataFrame

In [23]:
num = feature_importance_df[feature_importance_df['Importance']  <= 1e-4]
less_important_features = num.Feature.tolist()
len(less_important_features)

28152

In [24]:
num = feature_importance_df[feature_importance_df['Importance']  >= 1e-4]
important_features = num.Feature.tolist()
len(important_features)

68

In [25]:
def find_indexes(main_list, search_list):
    return [i for i, x in enumerate(main_list) if x in search_list]

indexes_shap = find_indexes(feature_importance_df['Feature'].tolist(), important_features)
print(len(indexes_shap))

68


In [ ]:
feature_importance_dict = dict(zip(feature_importance_df['Feature'], feature_importance_df['Importance']))
feature_importance_dict

In [ ]:
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

#### First we want to remove all values that are less than and equal to zero and save them in a list

removed_pairs_features = []

filtered_pairs_features = [(key, value) for key, value in sorted_features if value > 0]
removed_pairs_features = [(key, value) for key, value in sorted_features if value <= 0]
filtered_pairs_features_numpy = np.array(filtered_pairs_features)
# print("Filtered key-value pairs:", filtered_pairs_features)
# print("Removed key-value pairs:", removed_pairs_features)

In [ ]:
import matplotlib.pyplot as plt

# Extract feature names and importance values for visualization
sorted_feature_names = [x[0] for x in filtered_pairs_features[:20]]
sorted_feature_importance = [x[1] for x in filtered_pairs_features[:20]]

# Plot the ranked features
plt.figure(figsize=(10, 6))
plt.barh(sorted_feature_names, sorted_feature_importance)
plt.xlabel('SHAP Feature Importance')
plt.ylabel('Feature')
plt.title('Ranked Features based on SHAP Values')
plt.show()

In [ ]:
feature_shap_values = [pair[1] for pair in filtered_pairs_features[:30]]
feature_shap_values_names = [pair[0] for pair in filtered_pairs_features[:30]]
feature_names = np.array(feature_shap_values_names)
type(feature_shap_values)

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a list of features and their corresponding SHAP values
features = feature_importance_df['Feature'][:20]
shap_values = feature_importance_df['Importance'][:20]

# Plotting
plt.figure(figsize=(8, 6))
plt.scatter(shap_values, features, color='green', s=100, alpha=0.6)
plt.xlabel('SHAP Value')
plt.ylabel('Feature')
plt.title('Feature with their corresponding SHAP value')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
### Generating the FORCE Plot
shap_values = np.array(feature_shap_values)  # Assuming 100 samples and 5 features
shap_values_ = np.reshape(shap_values, (-1,2))
shap.force_plot(base_values, shap_values, feature_names)

In [ ]:
### Generating the Waterfall Plot

explanation = shap.Explanation(values=shap_values , base_values=base_values, feature_names = feature_names)

shap.waterfall_plot(explanation)

In [ ]:
genotype

In [ ]:
# Sample dataset with a categorical column
data = genotype
df_data = pd.DataFrame(data)
# Perform one-hot encoding on a specific categorical column
encoded_df = pd.get_dummies(df_data)

# Display the dataset after one-hot encoding
print("\nDataset after One-Hot Encoding:")
print(encoded_df)
encoded_df.shape

In [ ]:
import pandas as pd

# Sample one-hot encoded dataset
data = encoded_df
df_data = pd.DataFrame(data)

# Select features to set one-hot encoding to zero
selected_features = less_important_features

# Set one-hot encoding of selected features to zero
for feature in selected_features:
    for col in df_data.columns:
        if feature in col:
            df_data[col] = 0

# Check the modified dataset
print(df_data)

In [ ]:
print(df_data['1082942_chrIII_39540_C_A'])

In [ ]:
X = df_data
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno
Y = multi_pheno.iloc[:, 2]#index=1 --> 1_Diamide_1
# Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones
print(corrupted_X['6810072_chrXI_231860_A_G'])

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
print(type(corrupted_X_onehot))
#print(corrupted_X['6810072_chrXI_231860_A_G'])
# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

In [ ]:
### **Retrain the model on the new dataset**

def model_build(pheno_name,num_hl,hl_list,hl_activation, out_activation, dropout_val,filters_, kernel_size_,stride_poolSize,Conv_Layers_Stride_Size):
    

    '''
    https://towardsdatascience.com/multi-output-model-with-tensorflow-keras-functional-api-875dd89aa7c6

    def DNN_build(num_hl,hl_list,hl_activation, out_activation, dropout_val):
    pheno_name = name of phenotype of interest
    num_hl = number of hidden layers
    hl_list = list of hidden layers
    hl_activation = hidden layer activation function
    out_activation = output layer activation function
    dropout_val = Dropout value
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    input_layer = tf.keras.Input(shape=(28220,3))#(3138, 28220, 3)
    
    shared_convL1 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation,input_shape = (28220,3))(input_layer)
    shared_convL1_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL1)
    
    shared_convL2 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation)(shared_convL1_max_pool)
    shared_convL2_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL2)
    
    shared_convLayer_Flatten = tf.keras.layers.Flatten()(shared_convL2_max_pool)
    initializer = tf.keras.initializers.HeNormal()
    kernel_regularizer_ = tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001)
    
    shared_hidden_layer1 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[0], activation  = hl_activation)(shared_convLayer_Flatten)
    shared_hidden_layer1_dp1 = tf.keras.layers.Dropout(dropout_val[0])(shared_hidden_layer1)

    #model 1
    model1_hidden_layer2 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[1], activation  = hl_activation)(shared_hidden_layer1_dp1)
    model1_hidden_layer2_dp2 = tf.keras.layers.Dropout(dropout_val[1])(model1_hidden_layer2)

    model1_hidden_layer3 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[2], activation  = hl_activation)(model1_hidden_layer2_dp2)
    model1_hidden_layer3_dp3 = tf.keras.layers.Dropout(dropout_val[2])(model1_hidden_layer3)

    model1_hidden_layer4 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[3], activation  = hl_activation)(model1_hidden_layer3_dp3)
    model1_hidden_layer4_dp4 = tf.keras.layers.Dropout(dropout_val[3])(model1_hidden_layer4)
    model1_hidden_layer4_dp4_fl = tf.keras.layers.Flatten()(model1_hidden_layer4_dp4)
    
    pheno_name_ = tf.keras.layers.Dense(units = 1, name = pheno_name, activation  = out_activation)(model1_hidden_layer4_dp4_fl)#1_CobaltChloride_1
    model = tf.keras.models.Model(inputs = input_layer, outputs = [pheno_name_])

    return model



In [ ]:
#ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy','mse'])#change metrics to MSE
def compile_model(pheno_name,model,loss_, learningRate, metrics_): #loss = 'sparse_categorical_crossentropy'
    '''
    def compile_model(DNN, loss_, learningRate, metrics_):
    DNN: the model
    loss_: the loss function
    learningRate: learning rate
    metrics_: metrics of interest ['accuracy', 'mse']
    '''

    # Specify the optimizer, and compile the model with loss functions for both outputs
    model.compile(
       #optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate),#0.00027705 
    optimizer =  tf.keras.optimizers.legacy.SGD(learning_rate=learningRate, momentum=0.01, nesterov=True, name="SGD"),
      loss = {
          pheno_name:loss_
      }, 
      metrics = [metrics_]#{'CobaltChloride_1':[metrics_],'CopperSulfate_1':[metrics_],'Diamide_1':[metrics_]}
      )#change metrics to MSE ##['accuracy','mse']
    return model

def runModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode,pheno_index):
    '''
    def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
    pheno_index: the index of the phenotype in y_train
    '''
    history = model.fit(
    x_train, 
    [y_train[:]],
    validation_data=(x_valid, [y_valid[:]]),
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )

    return history



In [ ]:
def runModel_all(pheno_name, pheno_index,hl_list, drop_list):
    model_mse = model_build(pheno_name,4,hl_list,'relu','linear',drop_list,8,7,2,3)#model initiallization
    model_mse = compile_model(pheno_name,model_mse,tf.keras.losses.MeanSquaredError(reduction="auto", name="mse"),
    0.00123695244,'mse')
    history_mse =  runModel(model_mse,0.30,32,400,15, "val_mse", "min",pheno_index)
    return history_mse,model_mse

In [ ]:
history_mse,model_mse = runModel_all('1_Diamide_1',2,[1464,608,264,146],[0.00,0.00,0.00,0.20])

In [ ]:
model_mse.save('important features/saved Models/1_Diamide_1.h5')

In [ ]:
from sklearn.metrics import mean_squared_error

# Make predictions on the validation set
y_pred = model_mse.predict(x_valid)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_valid, y_pred)
print("Mean Squared Error on the validation set:", mse)

# Make predictions on the tesing set
y_pred = model_mse.predict(x_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on the testing set:", mse)



In [26]:
my_features = genotype.columns.tolist()
len(my_features)

28220

In [27]:
#### This is for LIME
import lime
import lime.lime_tabular
model_mse = tf.keras.models.load_model('1_Diamide_1.h5')
print(type(genotype))
genotype_data = genotype.to_numpy()
class_names = ['Positive','Negative']
my_features = genotype.columns.tolist()

2024-07-02 10:06:07.252168: W tensorflow/c/c_api.cc:300] Operation '{name:'1_Diamide_1_1/bias/Assign' id:717 op device:{requested: '', assigned: ''} def:{{{node 1_Diamide_1_1/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](1_Diamide_1_1/bias, 1_Diamide_1_1/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


<class 'pandas.core.frame.DataFrame'>


In [28]:
explainer = lime.lime_tabular.RecurrentTabularExplainer(x_train, mode='regression', training_labels=None, feature_names=my_features, 
                        categorical_features=None, categorical_names=None, kernel_width=None, kernel=None, verbose=False, 
                        class_names=class_names, feature_selection='auto', discretize_continuous=True, discretizer='quartile', 
                        random_state=None)

In [29]:
i = 0
instance = x_test[i]
explanation = explainer.explain_instance(instance, model_mse.predict, num_features=len(my_features))

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
2024-07-02 10:13:55.332451: W tensorflow/c/c_api.cc:300] Operation '{name:'1_Diamide_1_1/BiasAdd' id:722 op device:{requested: '', assigned: ''} def:{{{node 1_Diamide_1_1/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](1_Diamide_1_1/MatMul, 1_Diamide_1_1/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-07-02 10:13:55.496297: W tensorflow/c/c_api.cc:300] Operation '{name:'total/Assign' id:793 op device:{requested: '', assigned: ''} def:{{{node total/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](total, total/Initializer/zeros)}}' was changed by setti

In [30]:
explanation.as_list()
lime_importance = dict(explanation.as_list())
lime_importance

{'33147_chrI_33147_G_T_t-20570 <= 0.00': -0.00041743942603658624,
 '33147_chrI_33147_G_T_t-3821 <= 0.00': -0.0003987647663634313,
 '33152_chrI_33152_T_C_t-20660 <= 0.00': -0.0003942982927100574,
 '33147_chrI_33147_G_T_t-12061 <= 0.00': -0.00038983052541640247,
 '0.00 < 33147_chrI_33147_G_T_t-15621 <= 1.00': -0.0003873881384083983,
 '33147_chrI_33147_G_T_t-19066 <= 0.00': -0.0003863147559750956,
 '33147_chrI_33147_G_T_t-13572 <= 0.00': -0.0003800493376467037,
 '0.00 < 33147_chrI_33147_G_T_t-12931 <= 1.00': -0.0003750337918203069,
 '33152_chrI_33152_T_C_t-12658 <= 0.00': -0.0003710458687123554,
 '0.00 < 33152_chrI_33152_T_C_t-2638 <= 1.00': -0.000366896865265163,
 '0.00 < 33152_chrI_33152_T_C_t-19901 <= 1.00': -0.00036669399728179026,
 '33147_chrI_33147_G_T_t-20164 <= 0.00': -0.00036603975861007967,
 '0.00 < 33147_chrI_33147_G_T_t-9415 <= 1.00': -0.0003633883772626866,
 '33147_chrI_33147_G_T_t-22259 <= 0.00': -0.000362784548901238,
 '0.00 < 33152_chrI_33152_T_C_t-22463 <= 1.00': 0.000362

In [ ]:
sorted_lime_importance = dict(sorted(lime_importance.items(), key=lambda item: item[1], reverse=True))
sorted_lime_importance

In [ ]:
less_important_lime = [k for k, v in lime_importance.items() if v <= 1e-5]
len(less_important_lime)

In [31]:
important_lime = [k for k, v in lime_importance.items() if v >= 1e-5]
len(important_lime)

12308

In [34]:
def find_indexes(main_list, search_list):
    return [i for i, x in enumerate(main_list) if x in search_list]

indexes_lime = find_indexes(lime_importance, important_lime)
print(indexes_lime) 

[14, 23, 33, 34, 35, 38, 39, 44, 47, 49, 55, 56, 57, 59, 65, 70, 72, 76, 77, 83, 84, 85, 86, 88, 89, 90, 92, 97, 100, 102, 105, 110, 114, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 129, 135, 136, 139, 144, 148, 149, 150, 151, 152, 154, 158, 159, 161, 164, 165, 167, 172, 173, 174, 175, 177, 180, 184, 190, 191, 196, 197, 200, 203, 208, 210, 215, 218, 223, 224, 230, 233, 237, 241, 242, 245, 247, 248, 250, 251, 261, 262, 264, 265, 268, 273, 274, 277, 278, 279, 283, 287, 290, 294, 295, 296, 297, 298, 300, 303, 308, 314, 317, 318, 320, 323, 324, 330, 334, 336, 337, 339, 345, 347, 349, 352, 353, 356, 361, 365, 367, 368, 369, 370, 371, 372, 375, 378, 385, 388, 390, 392, 393, 394, 398, 402, 404, 406, 409, 411, 413, 426, 430, 433, 439, 441, 446, 449, 455, 458, 460, 463, 464, 467, 468, 469, 470, 471, 472, 474, 476, 478, 480, 482, 485, 488, 489, 490, 493, 494, 495, 496, 497, 498, 499, 501, 505, 506, 510, 511, 512, 517, 519, 532, 534, 538, 540, 543, 545, 548, 549, 550, 553, 559, 561, 56

In [ ]:
type(indexes)

In [ ]:
import matplotlib.pyplot as plt

sorted_lime_importance_list = sorted(sorted_lime_importance.items(), key=lambda item: item[1], reverse=True)

num_features_to_plot = min(20, len(sorted_lime_importance_list))

# Extract feature names and importance values for visualization
sorted_lime_names = [x[0] for x in sorted_lime_importance_list[:num_features_to_plot]]
sorted_lime_importance_ = [x[1] for x in sorted_lime_importance_list[:num_features_to_plot]]

# Plot the ranked features
plt.figure(figsize=(10, 6))
plt.barh(sorted_lime_names, sorted_lime_importance_)
plt.xlabel('LIME Feature Importance')
plt.ylabel('Feature')
plt.title('Ranked Features based on LIME Values')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

sorted_lime_importance_list = sorted(sorted_lime_importance.items(), key=lambda item: item[1], reverse=True)

num_features_to_plot = min(20, len(sorted_lime_importance_list))

# Extract feature names and importance values for visualization
sorted_lime_names = [x[0] for x in sorted_lime_importance_list[:num_features_to_plot]]
sorted_lime_importance_ = [x[1] for x in sorted_lime_importance_list[:num_features_to_plot]]
# Plotting
plt.figure(figsize=(8, 6))
plt.scatter(sorted_lime_importance_, sorted_lime_names, color='green', s=100, alpha=0.6)
plt.xlabel('LIME Value')
plt.ylabel('Feature')
plt.title('Feature with their corresponding SHAP value')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
# Sample dataset with a categorical column
data_lime = genotype
df_data = pd.DataFrame(data_lime)
# Perform one-hot encoding on a specific categorical column
encoded_df = pd.get_dummies(df_data)

# Display the dataset after one-hot encoding
print("\nDataset after One-Hot Encoding:")
print(encoded_df)
encoded_df.shape

In [ ]:
import pandas as pd

# Sample one-hot encoded dataset
data = encoded_df
df_data_ = pd.DataFrame(data)
type(df_data.columns)
# Select features to set one-hot encoding to zero
selected_features = indexes

# Set one-hot encoding of selected features to zero
for index in selected_features:
    for col in df_data_.columns:
        if index in df_data_.columns:
            df_data_[col] = 0

# Check the modified dataset
print(df_data_)

In [ ]:
print(df_data_['33070_chrI_33070_A_T'])

In [ ]:
X = df_data_
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno
Y = multi_pheno.iloc[:, 2]#index=1 --> 1_Diamide_1
# Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
print(type(corrupted_X_onehot))
#print(corrupted_X['6810072_chrXI_231860_A_G'])
# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

In [ ]:
### **Retrain the model on the new dataset**

def model_build(pheno_name,num_hl,hl_list,hl_activation, out_activation, dropout_val,filters_, kernel_size_,stride_poolSize,Conv_Layers_Stride_Size):
    

    '''
    https://towardsdatascience.com/multi-output-model-with-tensorflow-keras-functional-api-875dd89aa7c6

    def DNN_build(num_hl,hl_list,hl_activation, out_activation, dropout_val):
    pheno_name = name of phenotype of interest
    num_hl = number of hidden layers
    hl_list = list of hidden layers
    hl_activation = hidden layer activation function
    out_activation = output layer activation function
    dropout_val = Dropout value
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    input_layer = tf.keras.Input(shape=(28220,3))#(3138, 28220, 3)
    
    shared_convL1 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation,input_shape = (28220,3))(input_layer)
    shared_convL1_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL1)
    
    shared_convL2 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation)(shared_convL1_max_pool)
    shared_convL2_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL2)
    
    shared_convLayer_Flatten = tf.keras.layers.Flatten()(shared_convL2_max_pool)
    initializer = tf.keras.initializers.HeNormal()
    kernel_regularizer_ = tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001)
    
    shared_hidden_layer1 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[0], activation  = hl_activation)(shared_convLayer_Flatten)
    shared_hidden_layer1_dp1 = tf.keras.layers.Dropout(dropout_val[0])(shared_hidden_layer1)

    #model 1
    model1_hidden_layer2 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[1], activation  = hl_activation)(shared_hidden_layer1_dp1)
    model1_hidden_layer2_dp2 = tf.keras.layers.Dropout(dropout_val[1])(model1_hidden_layer2)

    model1_hidden_layer3 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[2], activation  = hl_activation)(model1_hidden_layer2_dp2)
    model1_hidden_layer3_dp3 = tf.keras.layers.Dropout(dropout_val[2])(model1_hidden_layer3)

    model1_hidden_layer4 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[3], activation  = hl_activation)(model1_hidden_layer3_dp3)
    model1_hidden_layer4_dp4 = tf.keras.layers.Dropout(dropout_val[3])(model1_hidden_layer4)
    model1_hidden_layer4_dp4_fl = tf.keras.layers.Flatten()(model1_hidden_layer4_dp4)
    
    pheno_name_ = tf.keras.layers.Dense(units = 1, name = pheno_name, activation  = out_activation)(model1_hidden_layer4_dp4_fl)#1_CobaltChloride_1
    model = tf.keras.models.Model(inputs = input_layer, outputs = [pheno_name_])

    return model



In [ ]:
#ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy','mse'])#change metrics to MSE
def compile_model(pheno_name,model,loss_, learningRate, metrics_): #loss = 'sparse_categorical_crossentropy'
    '''
    def compile_model(DNN, loss_, learningRate, metrics_):
    DNN: the model
    loss_: the loss function
    learningRate: learning rate
    metrics_: metrics of interest ['accuracy', 'mse']
    '''

    # Specify the optimizer, and compile the model with loss functions for both outputs
    model.compile(
       #optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate),#0.00027705 
    optimizer =  tf.keras.optimizers.legacy.SGD(learning_rate=learningRate, momentum=0.01, nesterov=True, name="SGD"),
      loss = {
          pheno_name:loss_
      }, 
      metrics = [metrics_]#{'CobaltChloride_1':[metrics_],'CopperSulfate_1':[metrics_],'Diamide_1':[metrics_]}
      )#change metrics to MSE ##['accuracy','mse']
    return model

def runModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode,pheno_index):
    '''
    def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
    pheno_index: the index of the phenotype in y_train
    '''
    history = model.fit(
    x_train, 
    [y_train[:]],
    validation_data=(x_valid, [y_valid[:]]),
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )

    return history



In [ ]:
def runModel_all(pheno_name, pheno_index,hl_list, drop_list):
    model_mse = model_build(pheno_name,4,hl_list,'relu','linear',drop_list,8,7,2,3)#model initiallization
    model_mse = compile_model(pheno_name,model_mse,tf.keras.losses.MeanSquaredError(reduction="auto", name="mse"),
    0.00123695244,'mse')
    history_mse =  runModel(model_mse,0.30,32,400,15, "val_mse", "min",pheno_index)
    return history_mse,model_mse

In [ ]:
history_mse,model_mse = runModel_all('1_Diamide_1',2,[1464,608,264,146],[0.00,0.00,0.00,0.20])

In [ ]:
from sklearn.metrics import mean_squared_error

# Make predictions on the validation set
y_pred = model_mse.predict(x_valid)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_valid, y_pred)
print("Mean Squared Error on the validation set:", mse)

# Make predictions on the tesing set
y_pred = model_mse.predict(x_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on the testing set:", mse)


In [ ]:
explanation.show_in_notebook(show_table=True)


In [ ]:
# intersection for both LIME and SHAP

In [35]:
def find_common_indexes(lime_indexes, shap_indexes):
    # Convert lists to sets
    lime_set = set(lime_indexes)
    shap_set = set(shap_indexes)
    
    # Find the intersection
    common_indexes = lime_set.intersection(shap_set)
    
    return common_indexes
    

common_indexes = find_common_indexes(indexes_lime, indexes_shap)
print(type(common_indexes))



<class 'set'>


In [36]:
common_indexes = np.array(list(common_indexes))
(common_indexes)

array([15234,  3078,  3084,  3089,  3090,  3095, 17610, 20559, 20560,
       20562, 20563, 20565, 10839, 20567, 20569, 20571, 17628, 20572,
       20575, 20576, 17634, 20578, 20580, 20581, 20582, 20583, 20586,
       10860, 20589, 20590, 15216, 20595, 10869, 20597,  3063, 20598,
       15229])

In [37]:
common_indexes_ = common_indexes.tolist()
len(common_indexes_)

37

In [38]:
def get_strings_at_indices(strings, indices):

    valid_indices = [index for index in indices if 0 <= index < len(strings)]
    
    # Get strings at valid indices
    result = [strings[index] for index in valid_indices]
    
    return result

matching_features = get_strings_at_indices(feature_importance_df['Feature'].tolist(), common_indexes_)
len(matching_features)

37

In [39]:
genotypeFile = 'genotype.csv'
genotype = pd.read_csv(genotypeFile, sep = '\t', index_col = 0)
print('genotypeFile shape:', genotype.shape )

phenotypeFile = 'phenotype.csv'
multi_pheno = pd.read_csv(phenotypeFile, sep = ',', index_col = 0)
print('Phenotype_Multi shape:', multi_pheno.shape )


# take a small part to test code
# genotype
X = genotype

mask = ~X.columns.str.contains('|'.join(matching_features))

# Set values to zero for non-matching columns
X.loc[:, mask] = 0

#display(X.head())
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno

#Chage the index for each model here accoding to the correspinding model
Y = multi_pheno.iloc[:, 2]#index=0 --> 1_Diamide_1
# # Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones
# corrupted_X.head()

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
# corrupted_X_onehot.shape

# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

genotypeFile shape: (4390, 28220)
Phenotype_Multi shape: (4390, 20)


In [40]:
### **Retrain the model on the new dataset**

def model_build(pheno_name,num_hl,hl_list,hl_activation, out_activation, dropout_val,filters_, kernel_size_,stride_poolSize,Conv_Layers_Stride_Size):
    

    '''
    https://towardsdatascience.com/multi-output-model-with-tensorflow-keras-functional-api-875dd89aa7c6

    def DNN_build(num_hl,hl_list,hl_activation, out_activation, dropout_val):
    pheno_name = name of phenotype of interest
    num_hl = number of hidden layers
    hl_list = list of hidden layers
    hl_activation = hidden layer activation function
    out_activation = output layer activation function
    dropout_val = Dropout value
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    input_layer = tf.keras.Input(shape=(28220,3))#(3138, 28220, 3)
    
    shared_convL1 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation,input_shape = (28220,3))(input_layer)
    shared_convL1_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL1)
    
    shared_convL2 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation)(shared_convL1_max_pool)
    shared_convL2_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL2)
    
    shared_convLayer_Flatten = tf.keras.layers.Flatten()(shared_convL2_max_pool)
    initializer = tf.keras.initializers.HeNormal()
    kernel_regularizer_ = tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001)
    
    shared_hidden_layer1 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[0], activation  = hl_activation)(shared_convLayer_Flatten)
    shared_hidden_layer1_dp1 = tf.keras.layers.Dropout(dropout_val[0])(shared_hidden_layer1)

    #model 1
    model1_hidden_layer2 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[1], activation  = hl_activation)(shared_hidden_layer1_dp1)
    model1_hidden_layer2_dp2 = tf.keras.layers.Dropout(dropout_val[1])(model1_hidden_layer2)

    model1_hidden_layer3 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[2], activation  = hl_activation)(model1_hidden_layer2_dp2)
    model1_hidden_layer3_dp3 = tf.keras.layers.Dropout(dropout_val[2])(model1_hidden_layer3)

    model1_hidden_layer4 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[3], activation  = hl_activation)(model1_hidden_layer3_dp3)
    model1_hidden_layer4_dp4 = tf.keras.layers.Dropout(dropout_val[3])(model1_hidden_layer4)
    model1_hidden_layer4_dp4_fl = tf.keras.layers.Flatten()(model1_hidden_layer4_dp4)
    
    pheno_name_ = tf.keras.layers.Dense(units = 1, name = pheno_name, activation  = out_activation)(model1_hidden_layer4_dp4_fl)#1_CobaltChloride_1
    model = tf.keras.models.Model(inputs = input_layer, outputs = [pheno_name_])

    return model



In [41]:
#ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy','mse'])#change metrics to MSE
def compile_model(pheno_name,model,loss_, learningRate, metrics_): #loss = 'sparse_categorical_crossentropy'
    '''
    def compile_model(DNN, loss_, learningRate, metrics_):
    DNN: the model
    loss_: the loss function
    learningRate: learning rate
    metrics_: metrics of interest ['accuracy', 'mse']
    '''

    # Specify the optimizer, and compile the model with loss functions for both outputs
    model.compile(
       #optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate),#0.00027705 
    optimizer =  tf.keras.optimizers.legacy.SGD(learning_rate=learningRate, momentum=0.01, nesterov=True, name="SGD"),
      loss = {
          pheno_name:loss_
      }, 
      metrics = [metrics_]#{'CobaltChloride_1':[metrics_],'CopperSulfate_1':[metrics_],'Diamide_1':[metrics_]}
      )#change metrics to MSE ##['accuracy','mse']
    return model

def runModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode,pheno_index):
    '''
    def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
    pheno_index: the index of the phenotype in y_train
    '''
    history = model.fit(
    x_train, 
    [y_train[:]],
    validation_data=(x_valid, [y_valid[:]]),
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )

    return history



In [42]:
def runModel_all(pheno_name, pheno_index,hl_list, drop_list):
    model_mse = model_build(pheno_name,4,hl_list,'relu','linear',drop_list,8,7,2,3)#model initiallization
    model_mse = compile_model(pheno_name,model_mse,tf.keras.losses.MeanSquaredError(reduction="auto", name="mse"),
    0.00123695244,'mse')
    history_mse =  runModel(model_mse,0.30,32,400,15, "val_mse", "min",pheno_index)
    return history_mse,model_mse

In [44]:
history_mse,model_mse = runModel_all('1_Diamide_1',2,[1464,608,264,146],[0.00,0.00,0.00,0.20])

Train on 3450 samples, validate on 385 samples


2024-07-02 10:27:31.900816: W tensorflow/c/c_api.cc:300] Operation '{name:'training_2/SGD/dense_6/kernel/momentum/Assign' id:2378 op device:{requested: '', assigned: ''} def:{{{node training_2/SGD/dense_6/kernel/momentum/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training_2/SGD/dense_6/kernel/momentum, training_2/SGD/dense_6/kernel/momentum/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Epoch 1/400
3360/3450 [============================>.] - ETA: 0s - loss: 17.7925 - mse: 0.1040

2024-07-02 10:27:34.106507: W tensorflow/c/c_api.cc:300] Operation '{name:'loss_2/AddN_1' id:2125 op device:{requested: '', assigned: ''} def:{{{node loss_2/AddN_1}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss_2/mul, loss_2/AddN)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


3450/3450 [==============================] - 2s 551us/sample - loss: 17.7903 - mse: 0.1020 - val_loss: 17.7002 - val_mse: 0.0194
Epoch 2/400
3450/3450 [==============================] - 2s 436us/sample - loss: 17.6978 - mse: 0.0243 - val_loss: 17.6857 - val_mse: 0.0197
Epoch 3/400
3450/3450 [==============================] - 2s 444us/sample - loss: 17.6818 - mse: 0.0231 - val_loss: 17.6705 - val_mse: 0.0192
Epoch 4/400
3450/3450 [==============================] - 2s 443us/sample - loss: 17.6672 - mse: 0.0233 - val_loss: 17.6559 - val_mse: 0.0194
Epoch 5/400
3450/3450 [==============================] - 1s 433us/sample - loss: 17.6521 - mse: 0.0229 - val_loss: 17.6410 - val_mse: 0.0193
Epoch 6/400
3450/3450 [==============================] - 1s 431us/sample - loss: 17.6370 - mse: 0.0226 - val_loss: 17.6262 - val_mse: 0.0193
Epoch 7/400
3450/3450 [==============================] - 1s 433us/sample - loss: 17.6217 - mse: 0.0221 - val_loss: 17.6114 - val_mse: 0.0192
Epoch 8/400
3450/3450 [==

In [45]:
from sklearn.metrics import mean_squared_error

# Make predictions on the validation set
y_pred = model_mse.predict(x_valid)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_valid, y_pred)
print("Mean Squared Error on the validation set:", mse)

# Make predictions on the tesing set
y_pred = model_mse.predict(x_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on the testing set:", mse)


`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
2024-07-02 10:28:48.583793: W tensorflow/c/c_api.cc:300] Operation '{name:'1_Diamide_1_3/BiasAdd' id:1976 op device:{requested: '', assigned: ''} def:{{{node 1_Diamide_1_3/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](1_Diamide_1_3/MatMul, 1_Diamide_1_3/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Mean Squared Error on the validation set: 0.01931313148307935
Mean Squared Error on the testing set: 0.01726684707710362
